# Interactive Databricks Table Editor with ipyaggrid

This notebook demonstrates how to create an interactive editor for Databricks tables using ipyaggrid with custom editors for different column types.

## Install Required Packages

First, let's install the necessary packages:

In [ ]:
%pip install ipyaggrid==0.3.0 pandas==1.5.3 pyspark==3.4.0

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
from ipyaggrid import Grid
import ipywidgets as widgets
from IPython.display import display, Javascript, HTML
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

## Configure Spark Session

In [ ]:
# Get existing SparkSession
spark = SparkSession.builder.getOrCreate()

## Define Table Configuration

In [ ]:
# Table configuration
TABLE_NAME = "default.your_table_name"  # Replace with your table name

# Define which columns to display and their types
COLUMN_CONFIG = {
    "id": {"editable": False},  # Non-editable column (primary key)
    "name": {"editable": True},  # Simple editable text field
    "status": {  # Dropdown selection
        "editable": True,
        "cellEditor": "agSelectCellEditor",
        "cellEditorParams": {
            "values": ["Active", "Inactive", "Pending", "Completed"]
        }
    },
    "priority": {  # Another dropdown
        "editable": True,
        "cellEditor": "agSelectCellEditor",
        "cellEditorParams": {
            "values": ["Low", "Medium", "High", "Critical"]
        }
    },
    "due_date": {  # Date picker
        "editable": True,
        "cellEditor": "agDateCellEditor",
        "valueFormatter": "d3.timeFormat('%Y-%m-%d')(new Date(data.due_date))"
    },
    "notes": {"editable": True}  # Simple editable text field
}

# List of columns to include
COLUMNS_TO_INCLUDE = list(COLUMN_CONFIG.keys())

## Data Loading Function

In [ ]:
def load_table_data():
    """
    Load data from Databricks table into a pandas DataFrame
    """
    # Read only the columns we need
    spark_df = spark.sql(f"SELECT {', '.join(COLUMNS_TO_INCLUDE)} FROM {TABLE_NAME}")
    
    # Convert to Pandas - limit rows if needed for better performance
    # pandas_df = spark_df.limit(1000).toPandas()  # Uncomment to limit rows
    pandas_df = spark_df.toPandas()
    
    # Convert date columns to string for better compatibility with ipyaggrid
    for col in pandas_df.columns:
        if pandas_df[col].dtype == 'datetime64[ns]':
            pandas_df[col] = pandas_df[col].dt.strftime('%Y-%m-%d')
    
    return pandas_df

## Configure Grid Column Definitions

In [ ]:
def create_column_defs():
    """
    Create column definitions for ipyaggrid based on config
    """
    column_defs = []
    
    for col_name, col_config in COLUMN_CONFIG.items():
        # Start with basic column definition
        col_def = {
            "headerName": col_name.replace('_', ' ').title(),
            "field": col_name,
            "editable": col_config.get("editable", False),
            "sortable": True,
            "filter": True
        }
        
        # Add special editor configurations if present
        if "cellEditor" in col_config:
            col_def["cellEditor"] = col_config["cellEditor"]
        
        if "cellEditorParams" in col_config:
            col_def["cellEditorParams"] = col_config["cellEditorParams"]
        
        if "valueFormatter" in col_config:
            col_def["valueFormatter"] = col_config["valueFormatter"]
        
        column_defs.append(col_def)
    
    return column_defs

## Create and Display the Grid

In [ ]:
# Load data
df = load_table_data()

# Create column definitions
column_defs = create_column_defs()

# Grid options configuration
grid_options = {
    "columnDefs": column_defs,
    "rowSelection": "multiple",
    "enableCellTextSelection": True,
    "ensureDomOrder": True,  # Important for maintaining edit state
    "rowMultiSelectWithClick": True,
    "rowDeselection": True,
    "pagination": True,
    "paginationPageSize": 20,
    "stopEditingWhenCellsLoseFocus": True,  # Important for capturing edits
    "enterMovesDown": False,  # Prevents automatic navigation when pressing Enter
    "enterMovesDownAfterEdit": False
}

# Create grid
grid = Grid(
    grid_data=df,
    grid_options=grid_options,
    column_defs=column_defs,
    theme="ag-theme-balham",  # or "ag-theme-alpine" for a different look
    columns_fit="auto",
    index=False,  # Hide index column
    quick_filter=True,  # Enable quick filtering
    export_excel=True,  # Enable Excel export
    export_csv=True,  # Enable CSV export
    height=500,  # Set grid height
    width="100%"  # Set grid width
)

# Create a status message widget to show edit status
status_widget = widgets.HTML(
    value="<div style='padding:10px;color:#666;'>Grid ready. Make changes and click 'Save Changes' when done.</div>"
)

# Create a button to save changes
save_button = widgets.Button(
    description='Save Changes',
    button_style='success',
    icon='save'
)

# Create a button to reload data
reload_button = widgets.Button(
    description='Reload Data',
    button_style='info',
    icon='refresh'
)

# Button container for layout
button_container = widgets.HBox([save_button, reload_button])

# Display all widgets
display(button_container, grid, status_widget)

## Handle Grid Events and Save Changes

In [ ]:
# Variable to store edited data
edited_data = []

# Function to handle cell value changes
def on_cell_changed(change):
    try:
        if change and 'new' in change and change['new']:
            # Extract cell change information
            cell_change = change['new']
            
            # Add to edited data list
            edited_data.append(cell_change)
            
            # Update status message
            status_widget.value = f"<div style='padding:10px;color:#008800;'>Cell edited: {cell_change['field']} = {cell_change['value']} (row {cell_change['rowIndex']})</div>"
            
            # Print for debugging
            print(f"Cell edited: {cell_change}")
    except Exception as e:
        status_widget.value = f"<div style='padding:10px;color:#cc0000;'>Error capturing edit: {str(e)}</div>"
        print(f"Error: {str(e)}")

# Register the callback for cell value changes
grid.observe(on_cell_changed, names='cell_changed')

In [ ]:
# Function to save changes back to the Databricks table
def save_changes(button):
    try:
        status_widget.value = "<div style='padding:10px;color:#0066cc;'>Processing changes...</div>"
        
        # First, get the current grid data (this is the workaround for widget sync issues)
        grid_data = grid.get_grid_data()
        
        if len(edited_data) == 0:
            status_widget.value = "<div style='padding:10px;color:#cc6600;'>No changes detected to save.</div>"
            return
        
        # Create a pandas DataFrame from the grid data
        updated_df = pd.DataFrame(grid_data)
        
        # Convert pandas DataFrame to Spark DataFrame
        updated_spark_df = spark.createDataFrame(updated_df)
        
        # Create a temporary view of the updated data
        updated_spark_df.createOrReplaceTempView("updated_data_view")
        
        # CRITICAL: Implement appropriate update logic based on your table structure
        # This example uses a merge pattern assuming 'id' is the primary key
        # Adjust according to your specific table schema and requirements
        
        merge_sql = f"""
        MERGE INTO {TABLE_NAME} AS target
        USING updated_data_view AS source
        ON target.id = source.id
        WHEN MATCHED THEN UPDATE SET
            {', '.join([f'target.{col} = source.{col}' for col in COLUMNS_TO_INCLUDE if col != 'id' and COLUMN_CONFIG[col].get('editable', False)])}
        """
        
        # Execute the merge operation
        spark.sql(merge_sql)
        
        # Clear edited data list
        edited_data.clear()
        
        # Update status message
        status_widget.value = f"<div style='padding:10px;color:#008800;'>Changes saved successfully at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</div>"
        
    except Exception as e:
        status_widget.value = f"<div style='padding:10px;color:#cc0000;'>Error saving changes: {str(e)}</div>"
        print(f"Error saving changes: {str(e)}")

# Register button click handlers
save_button.on_click(save_changes)

In [ ]:
# Function to reload data
def reload_data(button):
    try:
        status_widget.value = "<div style='padding:10px;color:#0066cc;'>Reloading data...</div>"
        
        # Reload data from database
        new_df = load_table_data()
        
        # Update grid with new data
        grid.grid_data = new_df
        
        # Clear edited data list
        edited_data.clear()
        
        # Update status message
        status_widget.value = f"<div style='padding:10px;color:#008800;'>Data reloaded successfully at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</div>"
        
    except Exception as e:
        status_widget.value = f"<div style='padding:10px;color:#cc0000;'>Error reloading data: {str(e)}</div>"
        print(f"Error reloading data: {str(e)}")

# Register reload button handler
reload_button.on_click(reload_data)

## Improved Widget Sync with Custom JavaScript (Optional)

In [ ]:
# This cell adds custom JavaScript to help with synchronization
# It's optional but can help with browser-specific issues

sync_js = """
require(['base/js/namespace'], function(Jupyter) {
    // Force kernel to process pending messages after cell edits
    Jupyter.notebook.events.on('edit_mode.Cell', function() {
        setTimeout(function() {
            Jupyter.notebook.kernel.execute('import time; time.sleep(0.1)');  
        }, 100);
    });
    
    // Add helpful notification
    console.log('Grid synchronization helper loaded.');
});
"""

display(Javascript(sync_js))

# Add HTML to improve widget rendering
display(HTML("""
<style>
.ag-theme-balham .ag-cell-focus, .ag-theme-alpine .ag-cell-focus {
    border: 1px solid #2196F3 !important;
    outline: none;
}
.ag-theme-balham .ag-cell-edited, .ag-theme-alpine .ag-cell-edited {
    background-color: rgba(255, 235, 59, 0.1) !important;
}
</style>
"""))

## Troubleshooting Guide

If you encounter issues with the grid, try the following solutions:

### Widget Sync Issues
- **Problem**: Changes made in the grid aren't being captured by the Python backend
- **Solution**: Use the manual save approach implemented above, which retrieves the entire grid data

### Browser Compatibility
- Chrome and Firefox work best with ipyaggrid
- If using Safari, make sure to enable all cookies and JavaScript

### Date Picker Issues
- If date picker doesn't work, try using the text input with YYYY-MM-DD format

### Performance Issues
- For large tables, add limit to the data loading function
- Consider implementing server-side pagination

### Cell Edit Not Registering
- Click outside the cell after editing
- Press Enter after making changes
- Try clicking the 'Save Changes' button to force data retrieval